### 1_采集数据集 6 类（清理 代码 789AT 等重复行）
##### 日线行情、股票列表、备用列表、曾用名、IPO新股、上市公司基本信息

In [1]:
import tushare as ts
import pandas as pd

# 设置 Tushare Token
#ts.set_token('d262023f0a14f06bba9b6c14c6627fae3780289dd4c497312f4c0c29')
ts.set_token('6488998e7e15b3596e11308b921fdc726d3401cd4bde7958d3ba49cd')
pro = ts.pro_api()

In [3]:
# 获取并保存日线行情数据
def fetch_and_save_daily_data(pro):
    # 调用接口获取日线数据
    df = pro.daily(
        ts_code="",   # 股票代码，空字符串表示获取所有股票
        trade_date="", # 交易日期，空字符串表示获取所有日期
        start_date="", # 起始日期
        end_date="",   # 结束日期
        offset="",     # 数据偏移量
        limit=""       # 返回数据的条数
    )
    # 过滤掉以8或9开头的股票代码
    df_filtered = df[~df['ts_code'].str.startswith(('8', '9'))]
    # 获取最新的交易日期
    latest_date = df_filtered['trade_date'].max()
    # 过滤出最新交易日期的数据
    df_latest = df_filtered[df_filtered['trade_date'] == latest_date]
    # 保存为CSV文件
    file_name = f"日线行情{latest_date}.csv"
    df_latest.to_csv(file_name, index=False)
    print(f"日线行情数据形状: {df_latest.shape}")

# 获取并保存上市公司基础信息
def fetch_and_save_stock_company_data(pro, limit=5000):
    offset = 0
    all_data = []
    while True:
        # 分批获取上市公司基础信息
        df = pro.stock_company(
            limit=limit,
            offset=offset,
            fields=["ts_code", "chairman", "manager", "reg_capital", "province", 
                    "city", "website", "email", "business_scope", "employees", 
                    "introduction", "setup_date", "main_business"]
        )
        if df.empty:
            break
        all_data.append(df)
        offset += limit
    all_data_df = pd.concat(all_data, ignore_index=True)
    # 过滤掉以8开头的股票代码
    filtered_df = all_data_df[~all_data_df['ts_code'].str.startswith('8')]
    filtered_df.to_csv('上市公司基本信息.csv', index=False)
    print(f"上市公司基础信息形状: {filtered_df.shape}")

# 获取并保存股票曾用名数据
def fetch_and_save_namechange_data(pro, limit=5000):
    offset = 0
    all_data = []
    while True:
        # 分批获取股票曾用名数据
        df = pro.namechange(
            ts_code="",
            start_date="",
            end_date="",
            limit=limit,
            offset=offset
        )
        if df.empty:
            break
        all_data.append(df)
        offset += limit
    all_data_df = pd.concat(all_data, ignore_index=True)
    # 过滤掉以T、A、9、8、7开头的股票代码
    filtered_df = all_data_df[~all_data_df['ts_code'].str.startswith(('T', 'A', '9', '8', '7'))]
    # 按公告日期降序排序，并去除重复的股票代码，保留最新记录
    filtered_df = filtered_df.sort_values(by='ann_date', ascending=False).drop_duplicates(subset='ts_code', keep='first')
    filtered_df.to_csv('股票曾用名.csv', index=False)
    print(f"股票曾用名数据形状: {filtered_df.shape}")

# 获取并保存新股上市数据
def fetch_and_save_new_share_data(pro, limit=5000):
    offset = 0
    all_data = []
    while True:
        # 分批获取新股上市数据
        df = pro.new_share(
            start_date="",
            end_date="",
            limit=limit,
            offset=offset
        )
        if df.empty:
            break
        all_data.append(df)
        offset += limit
    all_data_df = pd.concat(all_data, ignore_index=True)
    # 过滤掉以8或9开头的股票代码
    filtered_df = all_data_df[~all_data_df['ts_code'].str.startswith(('8', '9'))]
    filtered_df.to_csv('IPO新股上市.csv', index=False)
    print(f"新股上市数据形状: {filtered_df.shape}")

# 执行所有数据获取和保存操作
fetch_and_save_daily_data(pro)
fetch_and_save_stock_company_data(pro)
fetch_and_save_namechange_data(pro)
fetch_and_save_new_share_data(pro)

# 新增功能: 拉取股票列表数据
df = pro.stock_basic(**{
    "ts_code": "",     # 股票代码
    "name": "",        # 股票名称
    "exchange": "",    # 交易所
    "market": "",      # 市场
    "is_hs": "",       # 是否沪深港通
    "list_status": "", # 上市状态
    "limit": "",       # 返回数据的条数
    "offset": ""       # 数据偏移量
}, fields=[
    "ts_code",
    "symbol",
    "name",
    "area",
    "industry",
    "market",
    "list_date",
    "act_name",
    "act_ent_type",
    "fullname",
    "enname",
    "exchange",
    "is_hs"
])
# 过滤掉以8开头的股票代码
df_filtered = df[~df['ts_code'].str.startswith('8')]
df_filtered.to_csv('股票列表.csv', index=False)
print(f"股票列表数据形状: {df_filtered.shape}")

# 新增功能: 拉取备用列表数据
df = pro.bak_basic(**{
    "trade_date": "",   # 交易日期
    "ts_code": "",      # 股票代码
    "limit": "",        # 返回数据的条数
    "offset": ""        # 数据偏移量
}, fields=[
    "trade_date",
    "ts_code",
    "industry",
    "area",
    "pe",
    "float_share",
    "total_share",
    "total_assets",
    "liquid_assets",
    "fixed_assets",
    "reserved",
    "eps",
    "bvps",
    "pb",
    "list_date",
    "undp",
    "per_undp",
    "rev_yoy",
    "profit_yoy",
    "gpr",
    "npr",
    "holder_num",
    "name"
])
# 过滤掉以8开头的股票代码
df_filt = df[~df['ts_code'].str.startswith('8')]
# 根据 ts_code 去重，保留最新的记录（按 trade_date 排序）
filt_df = df_filt.sort_values(by='trade_date', ascending=False).drop_duplicates(subset='ts_code', keep='first')
filt_df.to_csv('备用列表.csv', index=False)
print(filt_df.shape)
print(f"备用列表数据形状: {filt_df.shape}")

日线行情数据形状: (5103, 11)
上市公司基础信息形状: (5404, 13)
股票曾用名数据形状: (5409, 6)
新股上市数据形状: (1762, 12)
股票列表数据形状: (5118, 13)
(5119, 23)
备用列表数据形状: (5119, 23)


## 2.1_数据清理
#### 整合：合并已采集的数据集（未清洗）

In [6]:
# 定义文件列表
files = [
    '备用列表.csv',
    '股票列表.csv',
    '股票曾用名.csv',
    'IPO新股上市.csv',
    '上市公司基本信息.csv'
]

# 使用glob查找符合模式的日线行情文件
daily_file = glob.glob('日线行情*.csv')
if daily_file:
    files.extend(daily_file)
    
# 读取所有文件并设置ts_code作为索引，存储在一个字典中
dfs = {file: pd.read_csv(file, index_col='ts_code') for file in files}

# 合并所有DataFrame，按索引进行外连接
merged_df = pd.concat(dfs.values(), axis=1)

# 保存合并后的数据
merged_df.to_csv('基础数据_未清洗.csv')

## 2.2_数据清理(完成采集+预处理end)
#### 预处理：清理行/退市/去重/去68，提取列

In [10]:
import os
import glob
import pandas as pd
from datetime import datetime

# 获取当前日期并格式化为 'YYYYMMDD' 格式
current_date = datetime.now().strftime('%Y%m%d')

# 读取 CSV 文件
df = pd.read_csv('基础数据_未清洗.csv')

# 计算市值并四舍五入到一位小数，然后添加到新列
df['market_value'] = round(df['total_share'] * df['close'], 1)

# 仅保留以 '6', '3', '0' 开头的代码
filtered_df = df[df['ts_code'].str.startswith(('6', '3', '0')) & ~df['ts_code'].str.startswith('68')]

# 清理标签 'area'地区 或 'industry' 为空值的行
df_cleaned = filtered_df.dropna(subset=['area'])

# 去除重复的行，保留最后一个出现的
df_cleaned = df_cleaned.drop_duplicates(subset='ts_code', keep='last')

# 定义所需的列，并按照给定顺序排列
desired_columns = [
    'ts_code', 'trade_date', 'name', 'industry', 'fullname', 'enname', 'area', 'city', 
    'list_date', 'ipo_date','ann_date', 'close', 'market_value', 'change_reason', 'is_hs', 'act_name', 'act_ent_type', 
    'chairman', 'manager', 'setup_date', 'reg_capital', 'business_scope', 'employees', 'introduction', 
    'website', 'email', 'main_business', 'pe', 'float_share', 'total_share', 
    'total_assets', 'liquid_assets', 'fixed_assets', 'reserved', 'eps', 'bvps', 'pb', 
    'undp', 'per_undp', 'rev_yoy', 'profit_yoy', 'gpr', 'npr', 'holder_num'
]

# 提取所需列
df_final = df_cleaned[desired_columns]

# 保存结果到新的 CSV 文件，文件名包含当前日期
output_filename = f'基础数据_预处理{current_date}.csv'
df_final.to_csv(output_filename, index=False)
print(df_final.shape)

# # 删除当前路径下除新生成的 CSV 文件之外的其他 CSV 文件
# csv_files = glob.glob('*.csv')
# for file in csv_files:
#     if file != output_filename:
#         os.remove(file)

# print(f"已删除除 '{output_filename}' 之外的其他 CSV 文件")

(4528, 44)


## 3_特征检索（功能改进中....）

#### 3.1【经营范围】检索

In [4]:
import pandas as pd

# 读取CSV文件
datapath = '../data/基础数据_预处理20241021.csv'
df = pd.read_csv(datapath)

def contains_any_keywords(text, keywords):
    # 检查文本中是否包含任意一个关键词
    return any(keyword in text for keyword in keywords)

# 关键词列表
keywords = ["二手车"]

# 检索列
columns_to_check = ['introduction', 'main_business', 'business_scope']

# 创建一个布尔DataFrame来存储每列的匹配结果
matches = pd.DataFrame()

# 检查每一列是否包含任意一个关键词
for column in columns_to_check:
    matches[column + '_匹配数'] = df[column].fillna('').apply(lambda x: contains_any_keywords(x, keywords))

# 筛选结果：检索关键词数量 & 市值范围 > 50亿
filtered_df = df[
    (matches.sum(axis=1) >= len(keywords)) &
    (pd.to_numeric(df['TMC'], errors='coerce') <= 500)
]

# 输出结果的形状 & 所有行指定列
print("搜索相关的公司数:", filtered_df.shape[0], "家")
print(filtered_df.iloc[:, [0, 1, 2, 21]])

# 输出表格
#filtered_df.to_csv('二手车相关公司.csv', index=False)

搜索相关的公司数: 19 家
        ts_code  name industry  total_assets
183   002917.SZ   金奥博     化工原料         31.75
389   300805.SZ  电声股份     广告包装         19.71
501   000701.SZ  厦门信达     商贸代理        218.49
707   001317.SZ   三羊马       铁路         16.83
831   002627.SZ  三峡旅游     旅游景点         41.41
952   301262.SZ  海看股份     影视音像         40.77
1782  300074.SZ  华平股份     软件服务         15.37
1789  603598.SH  引力传媒      互联网         15.52
1963  300572.SZ  安车检测     电器仪表         27.86
2171  600051.SH  宁波联合      综合类         52.51
2223  000715.SZ  中兴商业       百货         26.25
2314  000554.SZ  泰山石油     石油贸易         15.17
2386  600169.SH  太原重工     工程机械        345.01
2483  000757.SZ  浩物股份     汽车服务         26.05
2859  000550.SZ  江铃汽车     汽车整车        298.75
3482  000421.SZ  南京公用     供气供热        154.56
3756  600735.SH   新华锦       服饰         22.32
3981  600704.SH  物产中大     仓储物流       2072.42
4074  002758.SZ  浙农股份      批发业        195.13


#### 3.2【名称】检索

In [1]:
import pandas as pd

df = pd.read_csv('../data/基础数据_预处理20241021.csv')  # 读取名为'基础数据_预处理.csv'的文件

keywords = ["东方"]  # 关键词匹配

# 计算文本中包含关键词的数量
def match_keywords(text, keywords):
    return sum(1 for keyword in keywords if keyword in text) 
df['股票名称匹配数'] = df['name'].fillna('').apply(lambda x: match_keywords(x, keywords))  # 在'name'列中匹配关键词并统计匹配数
filtered_df = df[df['股票名称匹配数'] > 0]  # 筛选匹配到至少一个关键字的行


print("搜索相关的公司数:", filtered_df.shape[0], "家") 
print(filtered_df.iloc[:, :5]) 
filtered_df.to_csv('东方字头.csv', index=False)  

搜索相关的公司数: 32 家
        ts_code  name industry          fullname area
23    600776.SH  东方通信     通信设备        东方通信股份有限公司   浙江
230   000962.SZ  东方钽业      小金属      宁夏东方钽业股份有限公司   宁夏
456   603606.SH  东方电缆     电气设备      宁波东方电缆股份有限公司   浙江
659   002819.SZ  东方中科     电器仪表  北京东方中科集成科技股份有限公司   北京
833   002611.SZ  东方精工     轻工机械    广东东方精工科技股份有限公司   广东
1042  000725.SZ  京东方Ａ      元器件     京东方科技集团股份有限公司   北京
1148  300379.SZ   东方通     软件服务     北京东方通科技股份有限公司   北京
1221  300166.SZ  东方国信     软件服务    北京东方国信科技股份有限公司   北京
1272  300081.SZ  恒信东方     文教休闲      恒信东方文化股份有限公司   北京
1564  300118.SZ  东方日升     电气设备     东方日升新能源股份有限公司   浙江
1910  002167.SZ  东方锆业      小金属    广东东方锆业科技股份有限公司   广东
2533  002545.SZ  东方铁塔     农药化肥      青岛东方铁塔股份有限公司   山东
2568  600327.SH   大东方     医疗保健   无锡商业大厦大东方股份有限公司   江苏
2705  000682.SZ  东方电子     电气设备        东方电子股份有限公司   山东
2744  600811.SH  东方集团     农业综合        东方集团股份有限公司  黑龙江
2931  300059.SZ  东方财富       证券      东方财富信息股份有限公司   上海
2935  300217.SZ  东方电热     家用电器    镇江东方电热科技股份有限公司   江苏
2990  000301.

In [52]:
# 单独获取日线行情，交易日17：00后更新
# fetch_and_save_daily_data(pro)

日线行情数据形状: (5100, 11)


In [2]:
import pandas as pd

# 读取两个CSV文件
df1 = pd.read_csv('基础数据_预处理20240829.csv')
df2 = pd.read_csv('日线行情*.csv')


# 确保两个数据集中的'ts_code'列的数据类型一致
df1['ts_code'] = df1['ts_code'].astype(str)
df2['ts_code'] = df2['ts_code'].astype(str)

# 使用merge函数根据'ts_code'列合并两个数据集，并添加_indicator列
merged_df = pd.merge(df1, df2[['ts_code', 'close']], on='ts_code', how='left', indicator=True)

# 筛选出df1中存在的股票代码
filtered_df = merged_df[merged_df['_merge'] == 'both'].copy()

# 删除_indicator列
filtered_df.drop(columns=['_merge'], inplace=True)

# 计算市值并四舍五入到小数点后两位，然后添加到新列
filtered_df['market_value'] = filtered_df.apply(lambda row: round(row['total_share'] * row['close'], 1), axis=1)

# 将筛选后的数据集保存到新的CSV文件
filtered_df.to_csv('ffile.csv', index=False)